Requirements: Create an account on [FRED's Website](https://fred.stlouisfed.org/docs/api/api_key.html) and create an API key. Store the key in a local .env file like this FRED_API_KEY=your_actual_key_here_12345

In [6]:
import os
from dotenv import load_dotenv
from fredapi import Fred
import yfinance as yf
import pandas as pd
import numpy as np
from ta.momentum import RSIIndicator
from ta.trend import MACD
from scipy.signal import savgol_filter

In [2]:
# This looks for the .env file and loads variable
load_dotenv()
api_key = os.getenv('FRED_API_KEY')
fred = Fred(api_key=api_key)

In [3]:
# Fetch FRED data 
# GDPC1 = GDP, PCEPILFE = Core Inflation, UNRATE = Unemployment, M2SL = M2 Money Stock
gdp = fred.get_series('GDPC1')
inflation = fred.get_series('PCEPILFE')
employment = fred.get_series('UNRATE')
money_supply = fred.get_series('M2SL')

# Combine data
macro_data = pd.DataFrame({
    'GDP': gdp,
    'Core Inflation': inflation,
    'Unemployment Rate': employment,
    'M2 Money Stock': money_supply
})

# Aligning the data to the same frequency (daily) and forward filling missing values
macro_data = macro_data.resample('D').ffill().dropna()

In [4]:
# Fetch S&P 500 data
tickers = ['^GSPC', '^VIX', 'SPY']
sp500 = yf.download(tickers, start='1993-01-29')
gscp_price = sp500['Close']['^GSPC']
vix_price = sp500['Close']['^VIX']
spy_volume = sp500['Volume']['SPY']

market_data = pd.DataFrame({
    'GSPC': gscp_price,
    'VIX': vix_price,
    'SPY Volume': spy_volume
}).dropna()

[*********************100%***********************]  3 of 3 completed


In [5]:
# Join data and save to CSV
master_data = market_data.join(macro_data, how='left')
master_data = master_data.ffill().dropna()
master_data.to_csv('data/master_data.csv')

In [7]:
# Load existing data
df = pd.read_csv('data/master_data.csv', index_col=0, parse_dates=True)
df.index.name = 'Date'

print(f"Original shape: {df.shape}")
print(f"Original columns: {list(df.columns)}")

Original shape: (8318, 7)
Original columns: ['GSPC', 'VIX', 'SPY Volume', 'GDP', 'Core Inflation', 'Unemployment Rate', 'M2 Money Stock']


In [9]:
# Create return features
df['Return'] = df['GSPC'].pct_change()
df['Return_5d'] = df['GSPC'].pct_change(5)
df['Return_20d'] = df['GSPC'].pct_change(20)

# Optional: Smoothed return using Savitzky-Golay filter
# Handle NaNs by filling with 0 temporarily for smoothing
df['Return_Smooth'] = savgol_filter(
    df['Return'].fillna(0), 
    window_length=21, 
    polyorder=3, 
    mode='interp'
)

In [11]:
# Calculate technical indicators
rsi_indicator = RSIIndicator(close=df['GSPC'], window=14)
df['RSI_14'] = rsi_indicator.rsi()

# ========================================
# Note: MACD can produce NaN values for the initial periods, which is expected. We will handle this in the modeling phase.
# ========================================
macd_indicator = MACD(
    close=df['GSPC'],
    window_fast=12,
    window_slow=26,
    window_sign=9
)
df['MACD_Line'] = macd_indicator.macd()
df['MACD_Signal'] = macd_indicator.macd_signal()
df['MACD_Hist'] = macd_indicator.macd_diff()

In [12]:
# Create macroeconomic features (YoY changes)
# GDP: quarterly YoY (4 quarters back)
df['GDP_YoY'] = df['GDP'].pct_change(4 * 91)  # approx 4 quarters in days

# Core Inflation: monthly YoY (12 months back)
df['Core_Inflation_YoY'] = df['Core Inflation'].pct_change(12 * 30)  # approx 12 months

# M2 Money Stock: monthly YoY
df['M2_YoY'] = df['M2 Money Stock'].pct_change(12 * 30)

# Note: Unemployment Rate is typically used as-is (level, not YoY)

In [13]:
# REGIME LABEL (placeholder for now)
df['Regime'] = pd.NA  # Will be filled later with Bull/Bear/Sideways

In [16]:
# Drawdown from peak
df['Peak'] = df['GSPC'].cummax()
df['Drawdown'] = (df['GSPC'] - df['Peak']) / df['Peak']

# VIX change
df['VIX_Change'] = df['VIX'].diff()
df['VIX_Change_5d'] = df['VIX'].diff(5)

In [17]:
# Drop initial rows with too many NaNs from indicators
df_clean = df.dropna(subset=['Return', 'RSI_14', 'MACD_Line'])

In [18]:
df_clean.to_csv('data/master_data_enhanced.csv')
print(f"\nEnhanced shape: {df_clean.shape}")
print(f"Enhanced columns: {list(df_clean.columns)}")
print(f"\nSample of new columns:")
print(df_clean[['GSPC', 'Return', 'RSI_14', 'MACD_Line', 'MACD_Signal', 'Regime']].tail(10))


Enhanced shape: (8293, 23)
Enhanced columns: ['GSPC', 'VIX', 'SPY Volume', 'GDP', 'Core Inflation', 'Unemployment Rate', 'M2 Money Stock', 'Return', 'Return_5d', 'Return_20d', 'Return_Smooth', 'RSI_14', 'MACD_Line', 'MACD_Signal', 'MACD_Hist', 'GDP_YoY', 'Core_Inflation_YoY', 'M2_YoY', 'Regime', 'Peak', 'Drawdown', 'VIX_Change', 'VIX_Change_5d']

Sample of new columns:
                   GSPC    Return     RSI_14  MACD_Line  MACD_Signal Regime
Date                                                                       
2026-02-02  6976.439941  0.005391  57.293679  25.543772    24.368897   <NA>
2026-02-03  6917.810059 -0.008404  50.152902  21.581137    23.811345   <NA>
2026-02-04  6882.720215 -0.005072  46.423623  15.431381    22.135352   <NA>
2026-02-05  6798.399902 -0.012251  38.932072   3.710932    18.450468   <NA>
2026-02-06  6932.299805  0.019696  52.140083   5.167429    15.793861   <NA>
2026-02-09  6964.819824  0.004691  54.702521   8.843860    14.403860   <NA>
2026-02-10  6941.81

In [24]:
# Load enhanced data
df = pd.read_csv('data/master_data_enhanced.csv', index_col=0, parse_dates=True)

# Remove rows with missing technical/macro indicators (keep Regime as placeholder)
df_clean = df.drop('Regime', axis=1).dropna()
df_clean['Regime'] = pd.NA

# Save final clean dataset
df_clean.to_csv('data/master_data_clean.csv')

print("=" * 70)
print("FINAL CLEAN DATASET")
print("=" * 70)
print(f"Shape: {df_clean.shape}")
print(f"Date range: {df_clean.index.min().date()} to {df_clean.index.max().date()}")
print(f"Trading days: {len(df_clean)}")
print(f"Years covered: {(df_clean.index.max() - df_clean.index.min()).days / 365.25:.1f}")
print(f"\nMissing values per column:")
print(df_clean.isnull().sum()[df_clean.isnull().sum() > 0])
print("\n✓ Dataset ready for regime labeling and modeling!")

FINAL CLEAN DATASET
Shape: (7954, 23)
Date range: 1994-07-11 to 2026-02-13
Trading days: 7954
Years covered: 31.6

Missing values per column:
Regime    7954
dtype: int64

✓ Dataset ready for regime labeling and modeling!


| Column             | Type     | Description                         | Source      |
| ------------------ | -------- | ----------------------------------- | ----------- |
| Date               | datetime | Trading date (index)                | -           |
| GSPC               | float    | S&P 500 closing price               | Yahoo       |
| VIX                | float    | VIX closing level                   | Yahoo       |
| SPY Volume         | int      | SPY trading volume                  | Yahoo       |
| GDP                | float    | Real GDP (billions, chained 2017 $) | FRED        |
| Core Inflation     | float    | Core PCE index                      | FRED        |
| Unemployment Rate  | float    | Unemployment rate (%)               | FRED        |
| M2 Money Stock     | float    | M2 money supply (billions $)        | FRED        |
| Return             | float    | Daily S&P 500 return                | Computed    |
| Return_5d          | float    | 5-day return                        | Computed    |
| Return_20d         | float    | 20-day return                       | Computed    |
| Return_Smooth      | float    | Smoothed daily return               | Computed    |
| RSI_14             | float    | 14-day RSI                          | Computed    |
| MACD_Line          | float    | MACD line                           | Computed    |
| MACD_Signal        | float    | MACD signal line                    | Computed    |
| MACD_Hist          | float    | MACD histogram                      | Computed    |
| GDP_YoY            | float    | YoY GDP growth rate                 | Computed    |
| Core_Inflation_YoY | float    | YoY inflation rate                  | Computed    |
| M2_YoY             | float    | YoY M2 growth rate                  | Computed    |
| Peak               | float    | Running max of S&P 500              | Computed    |
| Drawdown           | float    | % decline from peak                 | Computed    |
| VIX_Change         | float    | Daily VIX change                    | Computed    |
| VIX_Change_5d      | float    | 5-day VIX change                    | Computed    |
| Regime             | object   | Bull/Bear label (to be filled)      | Placeholder |

In [ ]:
# ========================================
# The following is to test the sentiment data access from Refinitiv, could be skipped for now
# ========================================

# Check if you have Refinitiv access
try:
    import refinitiv.data as rd
    
    # Try to open a session (requires Workspace Desktop running)
    rd.open_session()
    print("✓ Refinitiv Data Library accessible")
    
    # Test if you can get news headlines
    headlines = rd.get_news_headlines(
        query="S&P 500",
        count=5
    )
    print("✓ News headlines accessible")
    print(headlines)
    
    rd.close_session()
except Exception as e:
    print(f"✗ Refinitiv access issue: {e}")


KeyboardInterrupt: 

In [ ]:
import requests
from time import sleep

def get_alphavantage_sentiment(api_key, tickers=['SPY'], limit=1000):
    """
    Get news sentiment from AlphaVantage
    """
    url = "https://www.alphavantage.co/query"
    
    all_sentiment = []
    
    for ticker in tickers:
        params = {
            'function': 'NEWS_SENTIMENT',
            'tickers': ticker,
            'apikey': api_key,
            'limit': limit
        }
        
        response = requests.get(url, params=params)
        data = response.json()
        
        if 'feed' in data:
            for article in data['feed']:
                date = pd.to_datetime(article['time_published'][:8])
                
                # Extract sentiment for this ticker
                for sentiment_item in article.get('ticker_sentiment', []):
                    if sentiment_item['ticker'] == ticker:
                        all_sentiment.append({
                            'date': date,
                            'ticker': ticker,
                            'sentiment_score': float(sentiment_item.get('ticker_sentiment_score', 0)),
                            'sentiment_label': sentiment_item.get('ticker_sentiment_label', 'Neutral')
                        })
                        break
        
        sleep(12)  # AlphaVantage rate limit: 5 calls/min for free, premium allows more
    
    sentiment_df = pd.DataFrame(all_sentiment)
    
    # Aggregate to daily
    daily_sentiment = sentiment_df.groupby('date').agg({
        'sentiment_score': 'mean'
    }).rename(columns={'sentiment_score': 'AlphaVantage_Sentiment'})
    
    return daily_sentiment

# Use your premium key
API_KEY = "VJJRWG9F79UTFIJT"
sentiment = get_alphavantage_sentiment(API_KEY, tickers=['SPY'])

# Merge with your data
df = pd.read_csv('data/master_data_enhanced.csv', index_col=0, parse_dates=True)
df = df.join(sentiment, how='left')
df['AlphaVantage_Sentiment'] = df['AlphaVantage_Sentiment'].ffill()

df.to_csv('data/master_data_with_sentiment.csv')
print(f"Added sentiment for {df['AlphaVantage_Sentiment'].notna().sum()} days")


Added sentiment for 2982 days
